# Informer

this code can also experiment with different number of inputs

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from informer import Informer
from datetime import datetime, timedelta
import yfinance as yf
import matplotlib.pyplot as plt

## Loading and preparing data and parameters

In [5]:
prediction_length = 10
input_length = 48
end_date = datetime.today()
start_date = end_date - timedelta(days=365)  
apple_stock_price = yf.download("AAPL", start=start_date, end=end_date, interval="1d")["Adj Close"]

[*********************100%%**********************]  1 of 1 completed


In [6]:
apple_stock_price = apple_stock_price.reset_index()
apple_stock_price.columns = ['ds', 'y']

df = apple_stock_price

In [7]:
train_df = df[:-prediction_length]
test_df = df[-prediction_length:]

# Scale data
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_df[['y']])
test_scaled = scaler.transform(test_df[['y']])

## Data needs to be in specific format

In [8]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, input_length, prediction_length):
        self.data = data
        self.input_length = input_length
        self.prediction_length = prediction_length

    def __len__(self):
        return len(self.data) - self.input_length - self.prediction_length + 1

    def __getitem__(self, idx):
        x = self.data[idx:idx+self.input_length]
        y = self.data[idx+self.input_length:idx+self.input_length+self.prediction_length]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

train_dataset = TimeSeriesDataset(train_scaled, input_length, prediction_length)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

## Defining the model

In [9]:
class InformerModel(nn.Module):
    def __init__(self, input_size, output_size, feature_size, num_layers, num_heads, dropout):
        super(InformerModel, self).__init__()
        self.informer = Informer(enc_in=input_size, dec_in=input_size, c_out=output_size,
                                 seq_len=input_length, label_len=input_length//2, out_len=prediction_length,
                                 factor=5, d_model=feature_size, n_heads=num_heads, e_layers=num_layers,
                                 d_layers=num_layers, d_ff=512, dropout=dropout, attn='prob', 
                                 embed='timeF', freq='h', activation='gelu', output_attention=False)
        
    def forward(self, x_enc, x_dec):
        return self.informer(x_enc, x_dec)[0]

model = InformerModel(input_size=1, output_size=1, feature_size=64, num_layers=2, num_heads=4, dropout=0.1)

TypeError: Informer.__init__() got an unexpected keyword argument 'enc_in'

## Training loop

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        for x, y in train_loader:
            optimizer.zero_grad()
            dec_inp = torch.zeros_like(x[:,-prediction_length:,:])
            dec_inp = torch.cat([x[:,-prediction_length:,:], dec_inp], dim=1)
            output = model(x, dec_inp)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

train_model(model, train_loader, criterion, optimizer, epochs=10)

## Model predictions

In [ ]:
model.eval()
test_data = torch.tensor(test_scaled[-input_length:].reshape(1, input_length, 1), dtype=torch.float32)
dec_inp = torch.zeros_like(test_data[:,-prediction_length:,:])
dec_inp = torch.cat([test_data[:,-prediction_length:,:], dec_inp], dim=1)
with torch.no_grad():
    predictions = model(test_data, dec_inp).numpy().flatten()

## Unscale predictions

In [ ]:
predictions = scaler.inverse_transform(predictions.reshape(-1, 1)).flatten()

## Plot

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df['ds'], df['y'], label='Actual')
plt.plot(df['ds'].iloc[-prediction_length:], predictions, label='Predicted', color='red')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Informer Model Predictions')
plt.show()